In [ ]:
import os
import sys
import fnmatch
import zipfile
import xmltodict
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')

In [ ]:
csr_unis = faculty_affil.affiliation.unique()

In [ ]:
r1 = pd.read_csv('other_ranks/r1.csv',header=None)

In [ ]:
i = 0
for row in r1.itertuples():
    if not row[1] in csr_unis:
        print(row[1])
        i +=1
print(i)
r1 = list(r1[0])

In [ ]:
r1_faculty_affil = {k:v for k,v in faculty_affil.iteritems()}

In [ ]:
# get r1
filter_set = [1 if row[2] in r1 else 0 for row in faculty_affil.itertuples()]
fss = np.array(filter_set).astype(np.bool)
r1_faculty_affil = faculty_affil[fss]

# get alias
r1_unique = sorted(set([aliasdict.get(row[1],row[1]) for row in r1_faculty_affil.itertuples()]))
len(r1_unique),r1_faculty_affil.shape
r1_unique_dict = {k:1 for k in r1_unique}
affils_dict = {row[1]:row[2] for row in r1_faculty_affil.itertuples()} 


In [ ]:
if False:
    r1_unique_dict2 = {k:1 for k in r1_unique_2}
    new_names = []
    for name in r1_unique:
        if name not in r1_unique_dict2:
            new_names.append(name)

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
#r1_confs = pickle.load(open('old_version/r1_confs.pkl','rb'))
#r1_confs_dict = {_:1 for _ in r1_confs}

In [ ]:
Xr1 = scipy.sparse.dok_matrix((len(r1),n_confs))
xdict = {}
for paper in all_papers:
    tag,title, authors, conf, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    j = conf_idx[conf]
    if year < 2000:
        continue
    for a in authors:
        name = aliasdict.get(a,a)
        if name in affils_dict:
            uni = affils_dict[name]
            i = r1.index(uni)
            xdict[(i,j)] = 1
Xr1._update(xdict)


In [ ]:
filtered_confs = (Xr1.sum(0) > 20)



r1_confs = [conf for conf in all_venues if filtered_confs[0,conf_idx[conf]]]
print(len(r1_confs))
#print(r1_confs)
r1_conf_idx = {v:i for i,v in enumerate(r1_confs)}
r1_conf_idx_orig = [conf_idx[conf] for conf in all_venues if filtered_confs[0,conf_idx[conf]]]

r1_scores = Xr1[:,np.array(filtered_confs).reshape((-1))]
r1_scores = np.array(r1_scores.sum(0)).reshape((-1))


In [ ]:
r1_confs

In [ ]:
Xpaper = scipy.sparse.dok_matrix((len(r1_unique),len(r1_confs)))
xdict = {}
for paper in all_papers:
    tag,title, authors, conf, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    if conf not in r1_conf_idx:
        continue
    if year < 2000:
        continue
    j = r1_conf_idx[conf]
    for a in authors:
        name = aliasdict.get(a,a)
        if name in affils_dict:
            i = r1_unique.index(name)
            xdict[(i,j)] = 1 + xdict.get((i,j),0)
Xpaper._update(xdict)

In [ ]:
r1counts = np.squeeze(np.array(Xpaper.sum(0)))
'SIGIR' in r1_conf_idx
if False:
    indexs = []
    for name in new_names:
        indexs.append(r1_unique.index(name))
    DP = np.array(Xpaper.todense())
    for idx  in np.argsort(DP[np.array(indexs),:].sum(1))[::-1]:
        idxt = indexs[idx]
        print(r1_unique[idxt],DP[idxt].sum())

In [ ]:
#filtered_authors = np.squeeze(np.array(Xpaper.sum(1) > 15) & np.array(Xpaper.sum(1) < 500))
#Xpaper_da = Xpaper[filtered_authors,:]
Xpaper_da = Xpaper

In [ ]:
Xpaper_da

In [ ]:
i1,i2 = r1_conf_idx['RECOMB'],r1_conf_idx['EMNLP']
Xpaper2 = np.array(Xpaper_da.todense())
filters_tmp = (Xpaper2[:,i1] > 2) & (Xpaper2[:,i2] > 2)
fidx = np.where(filters_tmp > 0)[0]

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
clf = LatentDirichletAllocation(50,max_iter=150)

In [ ]:
what = clf.fit_transform(Xpaper_da.T)
what.shape

In [ ]:
whatorig = np.copy(what)
what = what#/np.linalg.norm(what,axis=0)
what = np.copy(whatorig)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
embed = TSNE(perplexity=16,n_iter=12000,metric='cosine',init='random') # cosine euclidean
ys = embed.fit_transform(what)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
CLASS_NUM = 32
km = MiniBatchKMeans(CLASS_NUM,n_init=3200,max_iter=450)#, max_iter=5000,max_no_improvement=None,reassignment_ratio=0.02)
km.fit(what)
s = metrics.silhouette_score(what,km.labels_,metric='cosine')
print('{}\t\t{}'.format(CLASS_NUM,s))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.scatter(ys[:,0],ys[:,1],c=km.labels_/CLASS_NUM,cmap='tab20c')


In [ ]:
clf_gold = np.load('clf_gold.pkl.npy')
years_per_conf = clf_gold.shape[0]//n_confs
max_clf = clf_gold.reshape((-1,years_per_conf))[:,-5:].mean(1)
filt_vec = np.squeeze(np.array(filtered_confs))
max_clf_filt = max_clf[filt_vec]


In [ ]:
vector_to_look_at = max_clf_filt  #max_clf_filt or r1_scores
for i in range(CLASS_NUM):
    tmp = [(vector_to_look_at[i],i) for i,v in enumerate(km.labels_ == i) if v ] 
    score_idx = sorted(tmp,reverse=True)
    print(i,len(score_idx))
    for s,idx in score_idx[:5]:
        print('\t',r1_confs[idx])
    #print('NEXT')
    #tmp = [(r1_scores[i],i) for i,v in enumerate(km.labels_ == i) if v ] 
    #score_idx = sorted(tmp,reverse=True)
    #for s,idx in score_idx[:5]:
    #    print('\t',r1_confs[idx])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
plt.figure(figsize=(25,25))
cmap = plt.get_cmap('tab20c_r')
#plt.subplot(1,2,1)
plt.scatter(ys[:,0],ys[:,1],c=[cmap(_) for _ in km.labels_/CLASS_NUM],s=0.35*r1_scores**2,lw=0.1,edgecolors='k')
for i in range(CLASS_NUM):
    tmp = [(vector_to_look_at[i],i) for i,v in enumerate(km.labels_ == i) if v ] 
    score_idx = sorted(tmp,reverse=True)
    #print(i)
    k = 3
    j = 0
    for s,idx in reversed(score_idx[:k]):
        #if s < 1.5:
        #    continue
        rv = np.random.randn(2)
        xr,yr = 1.5*(rv)#/np.linalg.norm(rv)
        text = plt.text(xr+ys[idx,0],3*(j-(k-1)/2)/(k-1)+ys[idx,1],r1_confs[idx],size='20',color=np.array(cmap(i/CLASS_NUM)),
                 ha='center',va='center',alpha=0.9,weight='bold')
        text.set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),
                       path_effects.Normal()])
        j+=1
        #print('\t',r1_confs[idx])
    #print()
plt.xticks([],[])
plt.yticks([],[])
plt.tight_layout()
plt.axis('off')
plt.savefig('tsne3.pdf')

In [ ]:
np.save('labels',km.labels_)
np.save('mask',np.squeeze(np.array(filtered_confs)))

In [ ]:
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
name_idx = {v:i for i,v in enumerate(all_authors)}
n_auths = len(all_authors)


In [ ]:
num_filt_confs = what.shape[0]
old_idx_to_small = {}
j = 0
for i in range(filt_vec.shape[0]):
    if filt_vec[i]:
        old_idx_to_small[i] = j
        j+=1

In [ ]:
num_auth_papers = np.zeros(n_auths)
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    if filt_vec[conf_idx[venue]]:
        for ai in range(n):
            i = name_idx[authors[ai]]
            num_auth_papers[i] += 1/n

In [ ]:
import scipy.sparse
Xauth = scipy.sparse.dok_matrix((n_auths,num_filt_confs))
xdict = {}
for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    if filt_vec[conf_idx[venue]]:
        j = old_idx_to_small[conf_idx[venue]]
        for ai in range(n):#zip(count_vecs[n],authors):
            i = name_idx[authors[ai]]
            xdict[(i,j)] = (1/n)/num_auth_papers[i] + xdict.get((i,j),0)
    #j = years_per_conf*conf_idx[venue] + (year-min_year)
    #for ai in range(n):#zip(count_vecs[n],authors):
    #    i = name_idx[authors[ai]]
    #    #xdict[(i,j)] = 1/n + xdict.get((i,j),0)
    #    xdict[(i,j)] = count_vecs[n][ai] + xdict.get((i,j),0)


Xauth._update(xdict)

Xauth = scipy.sparse.csr_matrix(Xauth)

In [ ]:
Xmapped = Xauth @ what
Xmapped_2d = Xauth @ ys

In [ ]:
ranks = pd.read_csv('other_ranks/ranks.csv')
top_k = 36
subplot = int(round(np.sqrt(top_k)))
min_v = ys.min(0)
max_v = ys.max(0)
plt.figure(figsize=(subplot*4,subplot*4))
plt.rcParams = plt.rcParamsDefault
plt.set_cmap('viridis')
for i in range(top_k):
    Uname = ranks.iloc[i,:].uni
    uni_faculty = faculty_affil[faculty_affil.affiliation == Uname] 
    uni_names = np.array(uni_faculty.name)
    uni_names = list(uni_names)
    cmu_scores = []
    for name in set([aliasdict.get(n, n) for n in uni_names]):
        if name in name_idx:
            loc = Xmapped_2d[name_idx[name],:]
            cmu_scores.append((loc))
    cmu_scores = np.squeeze(np.array(cmu_scores))
    plt.subplot(subplot,subplot,i+1)
    plt.hexbin(cmu_scores[:,0],cmu_scores[:,1],gridsize=12,extent=(min_v[0],max_v[0],min_v[1],max_v[1]),vmin=0,vmax=4)
    plt.title(Uname,color='k',weight='demibold')
    plt.xticks([],[])
    plt.yticks([],[])
    plt.tight_layout()

plt.tight_layout()
plt.savefig('fingers3.pdf')

In [ ]:
#pickle.dump(what,open('top_conf_embed.pkl','wb'))
#pickle.dump(filtered_confs,open('top_conf_ind.pkl','wb'))

In [ ]:
#pickle.dump(r1_scores,open('r1_scores.pkl','wb'))
#pickle.dump(r1_confs,open('r1_confs.pkl','wb'))

In [ ]:
cmu_uni = pd.read_csv('other_ranks/cmu_faculty.csv')
cmu_uni = cmu_uni.fillna('CSD')
#print(list(cmu_uni.name))
uni_names = list(cmu_uni.name)
uni_labels = list(cmu_uni.dept)
uni_labels_unique = list(set(uni_labels))
uni_labels_unique = ['CSD','RI','MLD',"LTI",'HCI','ISR','BIO']

cmu_scores = []
uni_colors = []
for name,d in set([(aliasdict.get(n, n),dept) for n,dept in zip(uni_names,uni_labels)]):
    if name in name_idx:
        loc = Xmapped[name_idx[name],:]
        cmu_scores.append((loc,name))
        uni_colors.append( uni_labels_unique.index(d))

In [ ]:
embed2 = np.vstack([_[0] for _ in cmu_scores])

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
embedder2 = TSNE(2,20,init='pca',n_iter=6500)
ys2 = embedder2.fit_transform(embed2)

In [ ]:
import matplotlib.colors
import matplotlib.patheffects as path_effects

nc = (len(uni_labels_unique))
cmap = plt.get_cmap('tab10')
plt.figure(figsize=(30,30))
plt.scatter(ys2[:,0],ys2[:,1],s=120,lw=1,edgecolors='k',c=cmap(np.array(uni_colors)/nc-0.0))
for i in range(ys2.shape[0]):
    rv = np.random.randn(2)
    xr,yr = 0.2*(rv)#/np.linalg.norm(rv)
    text= plt.text(ys2[i,0]+xr,yr+ys2[i,1],cmu_scores[i][1],size='14',color=cmap(uni_colors[i]/nc),
             horizontalalignment='center',verticalalignment='center',alpha=0.9,weight='bold')
    text.set_path_effects([path_effects.Stroke(linewidth=0.2, foreground='black'),
               path_effects.Normal()])


In [ ]:
import matplotlib.colors
import matplotlib.patheffects as path_effects
from cycler import cycler
#plt.style.use('Solarize_Light2') #bmh
plt.style.use('default')
plt.style.use('seaborn-white')
#plt.rcParams["axes.prop_cycle"] =  cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
plt.rcParams["axes.prop_cycle"] =  cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b',  '#bcbd22', '#17becf'])
nc = (len(uni_labels_unique))
cmap = plt.get_cmap('tab10')
plt.figure(figsize=(12,12))
colors_for_now = []
uni_colors = np.array(uni_colors)
legends = []

for i in [0,1,2,3,4,5,6]:
    #c=[cmap(uni_colors[i]/10+0.2) for j in uni_colors if j==i],
    lgd = plt.scatter(ys2[i == uni_colors,0],ys2[i == uni_colors,1],s=25,lw=0.2,edgecolors='k',label=uni_labels_unique[i])
    legends.append(lgd)
colors_for_now = [l.get_facecolor() for l in legends]
plt.legend(markerscale = 5,prop={'size': 24}) #

if True:
    for i in range(ys2.shape[0]):
        rv = np.random.randn(2)
        xr,yr = 0.15*(rv)#/np.linalg.norm(rv)
        def name(txt):
            txts = txt.split(' ')
            if txts[-1].isdigit():
                txts = txts[:-1]
            return txts[0][0] + ' ' + txts[-1]
        the_name = name(cmu_scores[i][1])
        text= plt.text(ys2[i,0]+xr,yr+ys2[i,1],the_name,size='12',color=colors_for_now[uni_colors[i]][0],
                 horizontalalignment='center',verticalalignment='center',alpha=0.8,weight='demibold')
        #text.set_path_effects([path_effects.Stroke(linewidth=0.05, foreground='black'),
        #           path_effects.Normal()])
plt.gca().axis('off')
plt.xticks([],[])
plt.yticks([],[])
plt.tight_layout()
plt.savefig('cmu_faculty4.pdf')